<a href="https://colab.research.google.com/github/Umerfarooq122/Text-Summarization-by-Fine-tuning-a-pre-trained-transformer-PEGASUS-from-hugging-face-library/blob/main/PEGASUS_on_Samsum%2C_PubMed_and_Arxiv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Text Summarization Using Pre-training with Extracted Gap-sentences for Abstractive Summarization (PEGASUS) from Huggingface**

#### **Environment Setup:**

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_metric py7zr -q
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate

In [ ]:
!pip install datasets --upgrade
!pip install evaluate --upgrade
!pip install rouge_score --upgrade
!pip install transformers --upgrade

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig, BigBirdPegasusForConditionalGeneration, PegasusForConditionalGeneration, PegasusTokenizer
from peft import PeftModel, LoraConfig
from datasets import load_dataset
from evaluate import load
from trl import SFTTrainer
import torch
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### **PEGASUS large:**

In [ ]:
model_large = "google/pegasus-large"
tokenizer_large = AutoTokenizer.from_pretrained("google/pegasus-large")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import os
# Set the environment variable to enable more verbose error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
#model_pega = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv")
model_pega = AutoModelForSeq2SeqLM.from_pretrained(model_large)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### **PubMED Dataset:**

In [ ]:
dataset_science = load_dataset("ccdv/pubmed-summarization", trust_remote_code=True)
dataset_science

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

##### **Splitting the Dataset:**

In [ ]:
split_lengths = [len(dataset_science[split]) for split in dataset_science]
split_lengths

[14732, 819, 818]

In [ ]:
print(f"Features: {dataset_science['train'].column_names}")

Features: ['id', 'dialogue', 'summary']


In [ ]:
print("\nArticle:")
print(dataset_science["train"][0]['article'])
print("\nAbstract:")
print(dataset_science["train"][0]['abstract'])



Article:
Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)

Abstract:
Amanda baked cookies and will bring Jerry some tomorrow.


##### **Test Running Raw Model:**

In [ ]:
dialogue = dataset_science["train"][1]["article"]
summary = dataset_science["train"][1]["abstract"]


In [ ]:
pipe = pipeline("summarization", model=model_pega, tokenizer=tokenizer_large, device=device)

In [ ]:

max_length = 4000 # Set a suitable maximum length for your input

 #Truncate dialogue if it exceeds max_length
if len(dialogue) > max_length:
    dialogue = dialogue[:max_length]



In [ ]:

dialogue


'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great'

In [ ]:
summary

'Olivia and Olivier are voting for liberals in this election. '

In [ ]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer_large, model = model_pega)

In [ ]:
pipe_out = pipe(dialogue)
pipe_outp = pipe_out[0]["summary_text"]
pipe_outp


Your max_length is set to 128, but your input_length is only 26. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)


"Olivia: Who's you voting for in this election? Oliver: Liberals as always .<n>Olivia: Me too!!<n>Oliver: Great ."

In [ ]:
pipe_out

[{'summary_text': ''}]

In [ ]:
#print(pipe_outp.replace(" . <n>", ".\n"))

In [ ]:
#summary

##### **Evaluating The Performance of Raw Model:**

In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_TEST_ds(dataset, metric, model, tokenizer, batch_size=16, device=device, column_text="article", column_summary="abstract"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))
    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
    return metric.compute()

In [ ]:

rouge_metric = load("rouge")

score = calculate_metric_on_TEST_ds(dataset_science["train"][0:10], rouge_metric, model_pega, tokenizer_large, batch_size=2, column_text="article", column_summary='abstract')


100%|██████████| 5/5 [00:09<00:00,  1.87s/it]


In [ ]:

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

rouge_dict = {rn: score[rn] for rn in rouge_names}
print(pd.DataFrame(rouge_dict, index=["pegasus"]))


           rouge1  rouge2    rougeL  rougeLsum
pegasus  0.007424     0.0  0.007242   0.007415


##### **Fine-Tuning PEGASUS:**

###### **Tokenizing The Text:**

In [ ]:
tokens = tokenizer_large(dialogue, return_tensors="pt").input_ids
num_tokens = tokens.shape[1]
print(num_tokens)

4845


In [ ]:
def convert_examples_to_features(example_batch):

  input_encodings = tokenizer_large(example_batch["dialogue"], max_length=1024, truncation=True)
  with tokenizer_large.as_target_tokenizer():
      target_encodings = tokenizer_large(example_batch["summary"], max_length=256, truncation=True)
  return {
        "input_ids": input_encodings["input_ids"],
        "attention_mask": input_encodings["attention_mask"],
        "labels": target_encodings["input_ids"]
    }
dataset_science_pt = dataset_science.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
#dataset_science_pt["train"][0]

In [ ]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer_large, model = model_pega)

In [ ]:
lora_config = LoraConfig(r = 8, target_modules = ["q_proj","o_proj","k_proj", "v_proj",
                                                  "gate_proj", "up_proj", "down_proj"],
                         task_type = "CAUSAL_LM",)

###### **Setting The Training Arguments:**

In [ ]:
from transformers import TrainingArguments, Trainer
trainer_args = TrainingArguments(
    output_dir="pegasus-science",
    num_train_epochs=5,  # Reduced for monitoring potential overfitting
    warmup_steps=500,
    per_device_train_batch_size=2,  # Increased batch size, if possible
    per_device_eval_batch_size=2,
    weight_decay=0.005,  # Adjusted for potential improvement
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=20,  # More frequent evaluations
    save_steps=1000,  # More frequent saves
    gradient_accumulation_steps=8,  # Reduced to speed up updates
    fp16=True  # Enable mixed precision if hardware supports it
)


###### **Training The Model:**

In [ ]:
trainer = SFTTrainer(model=model_pega, args=trainer_args,
                  processing_class=tokenizer_large, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_science_pt["train"],
                  eval_dataset=dataset_science_pt["validation"])

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: umer5764 (umer5764-the-city-university-of-new-york). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
20,3.038600,2.496256
40,2.869300,2.313674
60,2.566900,2.116806
80,2.332000,1.957790
100,2.164200,1.834884
120,1.981300,1.753219
140,1.899400,1.692867
160,1.835800,1.645342
180,1.781300,1.608661
200,1.741000,1.582273


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=285, training_loss=2.076021268074973, metrics={'train_runtime': 3229.5937, 'train_samples_per_second': 22.808, 'train_steps_per_second': 0.088, 'total_flos': 6.217592416980173e+16, 'train_loss': 2.076021268074973, 'epoch': 4.964169381107492})

###### **Evaluating The Performance:**

In [ ]:
score = calculate_metric_on_TEST_ds(dataset_science["test"], rouge_metric, model_pega, tokenizer_large, batch_size=2, column_text="article", column_summary="abstract")
rouge_dict = {rn: score[rn] for rn in rouge_names}
pd.DataFrame(rouge_dict, index=["pegasus-PubMed"])

100%|██████████| 410/410 [08:37<00:00,  1.26s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.018438,0.000304,0.018302,0.018338


###### **Saving The Model:**

In [ ]:
model_pega.save_pretrained("pegasus-science-model")

In [ ]:
tokenizer_large.save_pretrained("pegasus-science-tokenizer")

#### **Arxiv Dataset:**

##### **Loading Dataset:**

In [ ]:
data_arxiv = load_dataset("ccdv/arxiv-summarization", trust_remote_code=True)
data_arxiv

##### **Tokenizing Data:**

In [ ]:
def convert_examples_to_features(example_batch):

  input_encodings = tokenizer_large(example_batch["article"], max_length=1024, truncation=True)
  with tokenizer_large.as_target_tokenizer():
      target_encodings = tokenizer_large(example_batch["abstract"], max_length=256, truncation=True)
  return {
        "input_ids": input_encodings["input_ids"],
        "attention_mask": input_encodings["attention_mask"],
        "labels": target_encodings["input_ids"]
    }
dataset_arxiv_pt = data_arxiv.map(convert_examples_to_features, batched=True)

##### **Setting the training parameters:**

In [ ]:
trainer_args = TrainingArguments(
    output_dir="pegasus-science",
    num_train_epochs=5,  # Reduced for monitoring potential overfitting
    warmup_steps=500,
    per_device_train_batch_size=2,  # Increased batch size, if possible
    per_device_eval_batch_size=2,
    weight_decay=0.005,  # Adjusted for potential improvement
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=20,  # More frequent evaluations
    save_steps=1000,  # More frequent saves
    gradient_accumulation_steps=8,  # Reduced to speed up updates
    fp16=True  # Enable mixed precision if hardware supports it
)

##### **Training The Model:**

In [ ]:
Arxtrainer = SFTTrainer(model=model_pega, args=trainer_args,
                  processing_class=tokenizer_large, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_arxiv_pt["train"],
                  eval_dataset=dataset_arxiv_pt["validation"])

In [ ]:
Arxtrainer.train()

##### **Evaluating The Performance:**

In [ ]:
score = calculate_metric_on_TEST_ds(data_arxiv["test"], rouge_metric, model_pega, tokenizer_large, batch_size=2, column_text="article", column_summary="abstract")
rouge_dict = {rn: score[rn] for rn in rouge_names}
pd.DataFrame(rouge_dict, index=["pegasus-Arxiv"])

#### **Samsum Dataset:**

##### **Loading Dataset:**

In [ ]:
data_samsum = load_dataset("Samsung/samsum", trust_remote_code=True)
data_samsum

##### **Tokenizing Data:**

In [ ]:
def convert_examples_to_features(example_batch):

  input_encodings = tokenizer_large(example_batch["dialogue"], max_length=1024, truncation=True)
  with tokenizer_large.as_target_tokenizer():
      target_encodings = tokenizer_large(example_batch["summary"], max_length=256, truncation=True)
  return {
        "input_ids": input_encodings["input_ids"],
        "attention_mask": input_encodings["attention_mask"],
        "labels": target_encodings["input_ids"]
    }
dataset_samsum_pt = data_samsum.map(convert_examples_to_features, batched=True)

##### **Training The Model:**

In [ ]:
samtrainer = SFTTrainer(model=model_pega, args=trainer_args,
                  processing_class=tokenizer_large, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"])

In [ ]:
samtrainer.train()

##### **Evaluating The Performance:**

In [ ]:
score = calculate_metric_on_TEST_ds(data_samsum["test"], rouge_metric, model_pega, tokenizer_large, batch_size=2, column_text="dialogue", column_summary="summary")
rouge_dict = {rn: score[rn] for rn in rouge_names}
pd.DataFrame(rouge_dict, index=["pegasus-Samsum"])

### **PEGASUS BigBird:**

#### **Loading The Model:**